In [1]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup
import re

In [7]:
def get_content(url):
    headers = {
        "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Encoding":"gzip, deflate, sdch",
        "Accept-Language":"ko-KR,ko;q=0.8,en-US;q=0.6,en;q=0.4",
        "Cache-Control":"max-age=0",
        "Connection":"keep-alive",
        "Cookie":"npic=eFBEBBU/yzEme0B5l5OcagDCMtaa5/G90XoG9Rkq9EP/wNGQGt+jPKJaBOveBEaFCA==; NNB=NLTF6ONPDQTFO; nx_ssl=2; ncvid=#vid#_14.39.150.134yORl; nci4=1e2acde1f7bcd6663bd59285887a057cdd867892c69d6fa17cb6ad5dd448b513813ffc27a629b6bcec4005d665c64d9647474a6bb086e7e8fc09e14d839409f9b7afa285a495d9a9a88faa9bd3a0b093b789c4b5bc9bbe8f978f82a286b7f6f2f3f2efededed9a95b097a9e4e08ce29a9bf28697f88994e7; nid_iplevel=1; page_uid=xERzTlpyLPZss5rLUkNssssssSh-406270; ncvc2=b4d42c085c6a6bdced107b5d7f9de2853a6a933c5f16c954b5704d9e2200e62df804c21bd9e570582abfe699; JSESSIONID=DD056961ED5BA6436D706429CBF42060; BMR=s=1464245784249&r=http%3A%2F%2Fm.cafe.naver.com%2FArticleRead.nhn%3Fclubid%3D10050146%26menuid%3D334%26articleid%3D319943171%26query%3D%25EB%25A7%25A5%25EB%25B6%2581&r2=http%3A%2F%2Fm.cafe.naver.com%2FArticleSearchList.nhn%3Fsearch.query%3D%25EB%25A7%25A5%25EB%25B6%2581%26search.menuid%3D%26search.searchBy%3D0%26search.sortBy%3Dsim%26search.clubid%3D10050146",
        "Host":"m.cafe.naver.com",
        "Referer":"http://m.cafe.naver.com/ArticleSearchList.nhn?search.query=%EB%A7%A5%EB%B6%81&search.menuid=&search.searchBy=0&search.sortBy=sim&search.clubid=10050146",
        "Upgrade-Insecure-Requests":"1",
        "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36",
    }
    response = requests.get(url, headers=headers)
    dom = BeautifulSoup(response.content, "html.parser")
    return dom.select_one("#postContent").text

In [8]:
def crawl_content(url):
    try:
        response = requests.get(url)
        dom = BeautifulSoup(response.content, "html.parser")
        return dom.select_one("#postContent").text
    except:
        return ""

In [9]:
def find_phonenumber(content):
    pattern = re.compile("[0-9O공영일이둘삼셋사넷오육칠팔구]{3}[.-]?[0-9O공영일이둘삼셋사넷오육칠팔구]{3,4}[.-]?[0-9O공영일이둘삼셋사넷오육칠팔구]{4}")
    phonenumbers = pattern.findall(content)
    if phonenumbers:
        return phonenumbers[0]
    return ""
def preprocess_phonenumber(phonenumber):
    return phonenumber.replace("삼","3").replace("O","0").replace("칠","7").replace("구","9")

In [10]:
url = "http://m.cafe.naver.com/ArticleSearchList.nhn?search.query=맥북&search.menuid=&search.searchBy=0&search.sortBy=sim&search.clubid=10050146"
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) " + \
            "Chrome/37.0.2062.94 Safari/537.36"
headers = {"User-Agent": user_agent}

response = requests.get(url, headers=headers)
dom = BeautifulSoup(response.content, "html.parser")
rows = dom.select("#articleList li")

df = pd.DataFrame(columns=["title","writer","no","date","link","content", "phonenumber"])

for row in rows:
    link = "http://m.cafe.naver.com" + row.select_one("a").get("href")
    title = row.select_one("h3").text.strip()
    writer = row.select_one(".name").text
    date = row.select_one(".time").text
    no = row.select_one(".no em").text
    content = get_content(link)
    phonenumber = preprocess_phonenumber(find_phonenumber(content))
    
    data_dict = {
        "title":title, 
        "writer":writer, 
        "no":no, 
        "date":date, 
        "link":link, 
        "content":content, 
        "phonenumber":phonenumber
    }

    df.loc[len(df)] = data_dict    
    
df

AttributeError: 'NoneType' object has no attribute 'text'